In [91]:
from os import access
import pandas as pd
import numpy as np
import datetime
from pandas.core.reshape.concat import concat
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from datetime import date, datetime

In [15]:
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
#Prepping EmployeeData (HVC_AM0)
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------


#Reading the file and storing each line in the list text
with open ("C:/Users/HP/Desktop/Dev.Maarten/Python/Project/Data/HVC_AM0.csv", "r") as f:
    textEmployees = f.readlines()
textEmployees.pop(0)

#We make a dictionnary based on the data
CompanyEmployeesDict = {}

for e in textEmployees:
    Employee_ID, EmployeeNumber, Name, Postcode, Location, Language = e.replace("\n","").split(";")
    Employee_ID=int(Employee_ID)
    Postcode=int(Postcode)
    CompanyEmployeesDict[Employee_ID]= {"EmployeeNumber":EmployeeNumber,"Name": Name, "Postcode": Postcode, "Location": Location,"Language": Language }


#Some Employees do not have a EmployeeNumber: We set the EmployeeNumber of these employees to 0
#In this way we can convert all to a float
for e in CompanyEmployeesDict:
    E_ID = CompanyEmployeesDict[e]["EmployeeNumber"]
    if(E_ID == ""):
        CompanyEmployeesDict[e]["EmployeeNumber"]="0"
    CompanyEmployeesDict[e]["EmployeeNumber"] = int(float(CompanyEmployeesDict[e]["EmployeeNumber"]))   

df_CompanyEmployees = pd.DataFrame.from_dict(CompanyEmployeesDict, orient = 'index')


#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
#Prepping ProductData (HVC_AR0)
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------


with open ("C:/Users/HP/Desktop/Dev.Maarten/Python/Project/Data/HVC_AR0.csv", "r") as f:
    textProducts = f.readlines()
textProducts.pop(0)

CompanyProductsDict = {}
for e in textProducts:
    ProductID, Description, Category, Family, Price = e.replace("\n","").split(";")
    ProductID = int(ProductID)
    CompanyProductsDict[ProductID]= {"Description":Description,"Category": Category, "Family": Family, "Price": Price}
   
#Some prices are missing   
for e in CompanyProductsDict:
  PriceCheck = CompanyProductsDict[e]["Price"]
  if(PriceCheck==""):
        CompanyProductsDict[e]["Price"]="0.0"
  CompanyProductsDict[e]["Price"]=float(CompanyProductsDict[e]["Price"])  

#Whenever they are Coupons they are not assigned a family
#We assign Coupon to its Family for clearity
for e in CompanyProductsDict:
  PriceCheck = CompanyProductsDict[e]["Family"]
  if(PriceCheck==""):
        CompanyProductsDict[e]["Family"]="Coupon"

df_Products = pd.DataFrame.from_dict(CompanyProductsDict, orient = 'index')


#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
#Prepping Reviews Data (HVC_CUSTOMER_REVIEWS)
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------   


with open ("C:/Users/HP/Desktop/Dev.Maarten/Python/Project/Data/HVC_CUSTOMER_REVIEWS.csv", "r") as f:
    textCustReviews = f.readlines()
textCustReviews.pop(0)

CompanyReviewsDict = {}
for e in textCustReviews:
    CustomerID, Review  = e.replace("\n","").split(";")
    CustomerID = int(CustomerID)
    CompanyReviewsDict[CustomerID]= {"Review": Review}

df_Reviews = pd.DataFrame.from_dict(CompanyReviewsDict, orient = 'index')


#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
#Prepping Depot data (HVC_DEPOT)
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------


#Reading the file and storing each line in the list text
with open ("C:/Users/HP/Desktop/Dev.Maarten/Python/Project/Data/HVC_DEPOT.csv", "r") as f:
    text_Depot = f.readlines()
text_Depot.pop(0)

#We make a dictionnary based on the data
Depot_Dict = {}

for dep in text_Depot:
    Depot_Id, Depot_Location, Depot_Route = dep.replace("\n","").split(";")
    Depot_Id = int(Depot_Id)
    Depot_Route = int(Depot_Route)
    Depot_Dict[Depot_Id] = {"DEPOT":Depot_Location, "HVROUTETEMPLATE_NRID":Depot_Route}

#The Depot_Id and HVROUTETEMPLATE_NRID are both integers, Depot_Location
#The names of the keys are based on the table descriptions for easy looking up in case of further calculations
#There are no missing values in our dictionnary so further preprocessing is not necessary.

df_Depot = pd.DataFrame.from_dict(Depot_Dict, orient = 'index')


#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
#Prepping DaysOfWeek (HVC_HVDAYOFWEEK)
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------


with open ("C:/Users/HP/Desktop/Dev.Maarten/Python/Project/Data/HVC_HVDAYOFWEEK.csv", "r") as f:
    text_DayOfWeek = f.readlines()
text_DayOfWeek.pop(0)

#We make a dictionnary based on the data
DayOfWeek_Dict = {}

for day in text_DayOfWeek:
    HVDAYOFWEEK_NRID, DAY = day.replace("\n","").split(";")
    HVDAYOFWEEK_NRID = int(float(HVDAYOFWEEK_NRID))
    DayOfWeek_Dict[int(HVDAYOFWEEK_NRID)] = DAY


#The HVDAYOFWEEK_NRID is an integer, DAY is a string. 
#!! important notice for further caclulations, the first day (day with id = 1) is a sunday! Not a monday!
df_DaysOfWeek = pd.DataFrame.from_dict(DayOfWeek_Dict, orient = 'index')


#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
#Prepping Position data (HVC_HVPOSITION)
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------


with open ("C:/Users/HP/Desktop/Dev.Maarten/Python/Project/Data/HVC_HVPOSITION.csv", "r") as f:
    text_Position = f.readlines()
text_Position.pop(0)

#We make a dictionnary based on the data
Position_Dict = {}

for loc in text_Position:
    SO0_NRID, LONG, LAT = loc.replace("\n","").split(";")
    SO0_NRID = int(float(SO0_NRID))
    LONG = float(LONG)
    LAT = float(LAT)
    Position_Dict[SO0_NRID] = {"LONG":LONG, "LAT":LAT}

#SO0_NRID is int, LONG and LAT are both float values.

df_Position = pd.DataFrame.from_dict(Position_Dict, orient = 'index')


#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
#Prepping Route Template data (HVC_HVROUTETEMPLATE)
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------


with open ("C:/Users/HP/Desktop/Dev.Maarten/Python/Project/Data/HVC_HVROUTETEMPLATE.csv", "r") as f:
    textRouteTemplate = f.readlines()
    textRouteTemplate.pop(0)
    
RouteTemplate_Dict = {}

for route in textRouteTemplate:
    HVROUTETEMPLATE_NRID, REGION, WEEKORDER, HVDAYOFWEEK_NRID = route.replace("\n","").split(";")
    HVROUTETEMPLATE_NRID=int(float(HVROUTETEMPLATE_NRID))
    RouteTemplate_Dict[HVROUTETEMPLATE_NRID] = {"REGION": REGION, "WEEKORDER": WEEKORDER, "DAYOFWEEK_NRID": HVDAYOFWEEK_NRID}        

df_RouteTemplate = pd.DataFrame.from_dict(RouteTemplate_Dict, orient = 'index')


#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
#Prepping Visit Outcome data (HVC_HVVISITOUTCOME)
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------


with open ("C:/Users/HP/Desktop/Dev.Maarten/Python/Project/Data/HVC_HVVISITOUTCOME.csv", "r") as f:
    textVisitOutcome = list()
    for line in f:
        line = line.replace("\n","")
        obs = line.split(";")
        textVisitOutcome.append(obs)

column_names = textVisitOutcome[0]
textVisitOutcome.pop(0)
VisitOucome_Dict = {}

for visit in textVisitOutcome:
    VisitOucome_Dict[visit[1]] = visit[0]


df_VisitOutcome = pd.DataFrame.from_dict(VisitOucome_Dict, orient = 'index')


#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
#Prepping TransactionData (HVC_HVVISITRESULT)
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------


#Reading the file and storing each line in the list text
with open ("C:/Users/HP/Desktop/Dev.Maarten/Python/Project/Data/HVC_HVVISITRESULT.csv", "r") as f:
    textVisit = f.readlines()
textVisit.pop(0)

#We make a dictionnary based on the data
CompanyVisitDict = {}

for e in textVisit:
    Visit_ID, Customer_ID, Employee_ID, VisitOutcome_ID, Time, Date, Amount, paymentMethod = e.replace("\n","").split(";")
    Visit_ID= int(Visit_ID)

    yYear, mMonth, dDay = Date.split("-")
    yYear = int(yYear)
    mMonth = int(mMonth)
    dDay = int(dDay)
    Date=datetime.date(yYear,mMonth,dDay)

    Customer_ID= int(Customer_ID)
    Employee_ID= int(Employee_ID)
    VisitOutcome_ID= int(VisitOutcome_ID)
    Amount= float(Amount)
    CompanyVisitDict[Visit_ID]= {"Customer_ID":Customer_ID,"Employee_ID": Employee_ID, "VisitOutcome_ID": VisitOutcome_ID, "Time": Time,"Date": Date, "Amount": Amount, "paymentMethod": paymentMethod }

#Some visits do not have a payment method recorded: We set the paymentMethod of these cases to the None variable
#Maybe these missing values will have value later on
for e in CompanyVisitDict:
    P_method = CompanyVisitDict[e]["paymentMethod"]
    if(P_method == ""):
        CompanyVisitDict[e]["paymentMethod"]= None

df_CompanyVisit = pd.DataFrame.from_dict(CompanyVisitDict, orient = 'index')

#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
#Prepping EmployeeData (HVC_SO0)
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------


#Reading the file and storing each line in the list text
with open ("C:/Users/HP/Desktop/Dev.Maarten/Python/Project/Data/HVC_SO0.csv", "r") as f:
    textCustomerType = f.readlines()
textCustomerType.pop(0)

#We make a dictionnary based on the data
CustomerTypeDict = {}

for e in textCustomerType:
    Customer_ID, RouteTemplate_ID, CustomerType, Postcode, Language, Season = e.replace("\n","").split(";")
    Customer_ID=int(float(Customer_ID))
    RouteTemplate_ID=int(float(RouteTemplate_ID))
    Postcode = int(float(Postcode))
    CustomerTypeDict[Customer_ID]= {"RouteTemplate_ID":RouteTemplate_ID,"CustomerType": CustomerType, "Postcode": Postcode, "Language": Language,"Season": Season}


df_CustomerType = pd.DataFrame.from_dict(CustomerTypeDict, orient = 'index')

#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
#Prepping ProductData (HVC_VISITRESULTDETAILS)
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------


with open ("C:/Users/HP/Desktop/Dev.Maarten/Python/Project/Data/HVC_VISITRESULTDETAILS.csv", "r") as f:
    textVisitDetails = f.readlines()
textVisitDetails.pop(0)


#We make a dictionnary based on the data
VisitDetailsDict = {}


for e in textVisitDetails:
    VisitDetails_ID, Product_ID, Quantity, Visit_ID = e.replace("\n","").split(";")
    VisitDetails_ID=int(VisitDetails_ID)
    Product_ID=int(Product_ID)
    Quantity=int(float(Quantity))
    Visit_ID=int(float(Visit_ID))
    VisitDetailsDict[VisitDetails_ID]= {"Product_ID":Product_ID,"Quantity": Quantity, "Visit_ID": Visit_ID}

df_VisitDetails = pd.DataFrame.from_dict(VisitDetailsDict, orient = 'index')

#We add a a selfmade database of average and median household income per postal code, the document is called Income_per_postcode2.csv
df_Income_per_postcode = pd.read_csv("C:/Users/HP/Desktop/Dev.Maarten/Python/Project/Data/Income_per_postcode2.csv")
#This dataframe may be important for several regressions questions 


#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#This gives us the following 11 dataframes to work with:
# 1  df_CompanyEmployees (HVC_AM0)
# 2  df_Products (HVC_AR0)
# 3  df_Reviews  (HVC_CUSTOMER_REVIEWS)
# 4  df_Depot  (HVC_DEPOT)
# 5  df_DaysOfWeek (HVC_HVDAYOFWEEK)
# 6  df_Position (HVC_HVPOSiTION)
# 7  df_RouteTemplate (HVC_HVROUTETEMPLATE)
# 8  df_VisitOutcome (HVC_HVVISITOUTCOME)
# 9  df_CompanyVisit (HVC_VISITRESULT)
# 10 df_CustomerType (HVC_SO0)
# 11 df_VisitDetails (HVC_VISITRESULTDETAILS)
# 12 df_Income_per_postcode (Income_per_postcode2, data source link: https://statbel.fgov.be/nl/themas/huishoudens/fiscale-inkomens#panel-13 )",
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
###########################
#Q1: What are the Products?
###########################

In [88]:
#Q1.1 What are the most frequently bought products?
Aureum = df_VisitDetails.groupby("Product_ID")["Quantity"].sum()
newAU = pd.merge(Aureum, df_Products, how="left", on=Aureum.keys())
Revenue = newAU["Price"]*newAU["Quantity"]
newAU["RevenueProduct"] = Revenue


#Most products sold in ammount
BestAmountSoldProducts = newAU.sort_values(by="Quantity", ascending=False)

In [ ]:
#Q1.2 Which products render the most revenue?
BestRevProducts = newAU.sort_values(by="RevenueProduct", ascending=False)

In [ ]:
#Q1.3 Which products are bought the most in the region of Brussels, Antwerp, …?

In [ ]:
#Q1.4 Are product purchases correlated? Are some products often purchased together?

In [ ]:
#Q1.5 What are the total sales generated for each product family?
#Based on Family
#Generated Revenue
BestRevFamily=BestRevProducts.groupby("Family")["RevenueProduct"].sum().sort_values(ascending=False)
#Based on Family
#Amount sold
BestAmountFamily=BestAmountSoldProducts.groupby("Family")["Quantity"].sum().sort_values(ascending=False)

#Based on Catogery
#Generated Revenue
BestRevCat=BestRevProducts.groupby("Category")["RevenueProduct"].sum().sort_values(ascending=False)

#Based on Catogery
#Amount sold
BestAmountCat=BestRevProducts.groupby("Category")["Quantity"].sum().sort_values(ascending=False)
print(BestAmountCat)

In [102]:
#Q1.6 Does the weather / seasonal changes have an effect on the total revenue?
#IMPORTANT This question requires the dataFrame newAU from 1.1
df_CompanyVisit

#This is used to define a function that calculates which season a given date is
#This function: get_season returns the season of a date
#We add this to a new Column Season
Y=2020
seasons = [('Winter', (date(Y,  1,  1),  date(Y,  3, 20))),
           ('Spring', (date(Y,  3, 21),  date(Y,  6, 20))),
           ('Summer', (date(Y,  6, 21),  date(Y,  9, 22))),
           ('Autumn', (date(Y,  9, 23),  date(Y, 12, 20))),
           ('Winter', (date(Y, 12, 21),  date(Y, 12, 31)))]

def get_season(now):
    if isinstance(now, datetime):
        now = now.date()
    now = now.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= now <= end)

df_CompanyVisit["Season"]=df_CompanyVisit["Date"].apply(get_season)
df_Seasonality=df_CompanyVisit.groupby("Season")["Amount"].sum()
#print(df_Seasonality.head(5))

In [ ]:
##########################
#Q2 Who are the customers?
##########################

In [ ]:
#Q2.1 What do customers buy?

In [101]:
#Q2.2 Which customers left the company?
#mostRecent= max(df_CompanyVisit["Date"]) - datetime.timedelta(0)
#mostRecentTest = mostRecent - datetime.timedelta(182)
#df_CompanyVisit["LastVisit"]= datetime.date.today() - df_CompanyVisit["Date"]
#def isOk(dateCh: datetime.date):
 #   if((mostRecent-dateCh)> (mostRecent-mostRecentTest)):
  #      return True
   # else:
    #    return False
#df_CompanyVisit["Churn?"]=df_CompanyVisit["Date"].apply(isOk)        
#print(df_CompanyVisit.head(5))


AttributeError: type object 'datetime.datetime' has no attribute 'timedelta'

In [7]:
#Q2.3 Which customers have the highest CLV?
df_OnlySuccesFullVisits= df_CompanyVisit[df_CompanyVisit["VisitOutcome_ID"] == 2]
df_OnlySuccesFullVisits= df_OnlySuccesFullVisits.sort_values("Customer_ID")
#We sum the amounts of money spent per Customer_ID and this gives us after sorting the CLV of all Customers in descending order
df_CLV=df_OnlySuccesFullVisits.groupby("Customer_ID")["Amount"].sum().sort_values(ascending=False)
#print(df_CLV.head(50))

In [ ]:
#Q2.4 What is the relationship between leaving the company and buying patterns?

In [ ]:
#Q2.5 Are there clusters of customers? How would you describe these different clusters?

In [99]:
#Q2.6 Do customers have different buying patterns during the weekend?

#This method uses the weekday function to determine the day of the week starting 
#At 0 for Monday and ending on 6 for Sunday for a given month and year
#In this way if the returned number is larger than 4 (Friday) it is a weekendDay
def getWeekIndex(d: datetime.date):
    if(d.weekday()>4):
        return "weekend"
    else:
        return "weekday"

#We add a new Column explaining wheter it is a weekday or weekend based on the Date index
df_CompanyVisit["DateIndex"] = df_CompanyVisit["Date"].apply(getWeekIndex)
df_CompanyVisit["Visit_ID"] = df_CompanyVisit.index

CompleteMerge = pd.merge(df_VisitDetails, df_CompanyVisit, on="Visit_ID")


df_Products["Product_ID"] = df_Products.index
CompleteMergeProd = pd.merge(CompleteMerge,df_Products, on="Product_ID").drop(columns=["paymentMethod", "Amount", "Time"])
CompleteMergeProd["Revenue"]=CompleteMergeProd["Quantity"]*CompleteMergeProd["Price"]

OnlyWeekdays= CompleteMergeProd[CompleteMergeProd["DateIndex"] != "weekend"]
OnlyWeekends= CompleteMergeProd[CompleteMergeProd["DateIndex"] != "weekday"]

OnlyWeekdaysCus = OnlyWeekdays.groupby(["Customer_ID","DateIndex"])["Revenue"].sum()
OnlyWeekendsCus = OnlyWeekends.groupby(["Customer_ID","DateIndex"])["Revenue"].sum()
Comparison = pd.merge(OnlyWeekendsCus,OnlyWeekdaysCus, on="Customer_ID")

#X is weekends Y is weekdays
#This shows the difference in Revenue expressed in mean and other statistics
#Comparing the weekend and 
#print(Comparison.describe())


OnlyWeekdaysPro =  OnlyWeekdays.groupby(["Product_ID","DateIndex"])["Revenue"].sum()
OnlyWeekendsPro =  OnlyWeekends.groupby(["Product_ID","DateIndex"])["Revenue"].sum()
OnlyWeekdaysProMerged = pd.merge(OnlyWeekdaysPro,df_Products, on="Product_ID").drop(columns="Price")
OnlyWeekendsProMerged =  pd.merge(OnlyWeekendsPro,df_Products, on="Product_ID").drop(columns="Price")

OnlyWeekdaysProMergedFam =  pd.merge(OnlyWeekdaysPro,df_Products, how="left",on="Product_ID").drop(columns="Price").groupby("Family")["Revenue"].sum()
OnlyWeekendsProMergedFam =  pd.merge(OnlyWeekendsPro,df_Products, how="left",on="Product_ID").drop(columns="Price").groupby("Family")["Revenue"].sum()

#X is weekends Y is weekdays
#This show what Family sells more during either weekend or weekday
WeekendComparedFam = pd.merge(OnlyWeekendsProMergedFam,OnlyWeekdaysProMergedFam, on="Family")
#print(WeekendComparedFam)

OnlyWeekdaysProMergedDes =  pd.merge(OnlyWeekdaysPro,df_Products, how="left",on="Product_ID").drop(columns="Price").groupby("Description")["Revenue"].sum()
OnlyWeekendsProMergedDes =  pd.merge(OnlyWeekendsPro,df_Products, how="left",on="Product_ID").drop(columns="Price").groupby("Description")["Revenue"].sum()

#X is weekends Y is weekdays
#This show which products sell more during either weekend or weekday
WeekendComparedDis= pd.merge(OnlyWeekendsProMergedDes,OnlyWeekdaysProMergedDes, on="Description")
#print(WeekendComparedDis)

In [86]:
#Q2.7 Do customer sales differ across different cities? Is there a relationship between customer
#sales and average income per inhabitant (and other factors)?
##Important!!! running this question requires running question 2.3 for the CLV dataframe
df_CustomerType["CLV"]=df_CLV
df_CustomerType = df_CustomerType.dropna(axis="rows")
df_RouteTemplate["RouteTemplate_ID"]=df_RouteTemplate.index


#This compares CLV over all regions merging over Routetempate to get CLV linked to The Region
df_CLV_Route = pd.merge(df_CustomerType,df_RouteTemplate, on="RouteTemplate_ID")
df_CLV_Route_Region = df_CLV_Route.groupby("REGION")["CLV"].sum()
#print(df_CLV_Route_Region)

#Compares CLV over all Cities
df_Depot=df_Depot.rename(columns={"HVROUTETEMPLATE_NRID": "RouteTemplate_ID"})
df_CLV_Cities= pd.merge(df_CLV_Route,df_Depot,on="RouteTemplate_ID").groupby("DEPOT")["CLV"].sum()
#df_CLV_Cities.head(5)

#df_Income_per_postcode
#Per postcode : cust type: language : season_type
lr_model = LinearRegression()
#We merge customertype with the new CSV income per postcode to get median income and average income
#Based on where the customer lives
df_featuresFrame = pd.merge(df_CustomerType,df_Income_per_postcode, how="right" ,on="Postcode")
df_featuresFrame=df_featuresFrame.dropna()

#We apply scalers to these paramaters (Also cat var??)
scaler = StandardScaler(df_featuresFrame)
df_featuresFrame["AVERAGE_INCOME"] = df_featuresFrame["AVERAGE_INCOME"].astype(float)
df_featuresFrame["MEDIAN_INCOME"] = df_featuresFrame["MEDIAN_INCOME"].astype(float)

#Encading cat var
label_encoder1 = LabelEncoder()
label_encoder1.fit(df_featuresFrame["CustomerType"])
def changeLan(lang):
    if(lang == "nl_BE"):
        return 0
    else:
        return 1    
df_featuresFrame["CustomerTypeNum"] = label_encoder1.transform(df_featuresFrame["CustomerType"])    

#0 is dutch 1 speaks french
df_featuresFrame["Language"]=df_featuresFrame["Language"].apply(changeLan)
print(df_featuresFrame.head(5))

scaler.fit(df_featuresFrame[["AVERAGE_INCOME","MEDIAN_INCOME","CustomerTypeNum","Language"]])


#I have no idea how to incorprate CUSTOMERTYPENUM and LANGUAGE because they are catogorical vars so idk if this is right
lr_model.fit(X=df_featuresFrame[["AVERAGE_INCOME","MEDIAN_INCOME","CustomerTypeNum","Language"]], y=df_featuresFrame["CLV"])
lr_model.coef_


   RouteTemplate_ID CustomerType  Postcode  Language    Season    CLV  \
0       289662608.0      Private      3500         1  All time  285.2   
1       289662608.0      Private      3500         1  All time  103.2   
2       289658600.0      Private      3500         1  All time  117.0   
3       289658600.0      Private      3500         1  All time   82.6   
4       289658600.0      Private      3500         1  All time  204.3   

  City_name  AVERAGE_INCOME  MEDIAN_INCOME  CustomerTypeNum  
0   Hasselt         35091.0        26519.0                2  
1   Hasselt         35091.0        26519.0                2  
2   Hasselt         35091.0        26519.0                2  
3   Hasselt         35091.0        26519.0                2  
4   Hasselt         35091.0        26519.0                2  
RouteTemplate_ID    float64
CustomerType         object
Postcode              int64
Language              int64
Season               object
CLV                 float64
City_name            

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass copy=      RouteTemplate_ID CustomerType  Postcode Language    Season      CLV  \
0          289662608.0      Private      3500    nl-BE  All time   285.20   
1          289662608.0      Private      3500    nl-BE  All time   103.20   
2          289658600.0      Private      3500    nl-BE  All time   117.00   
3          289658600.0      Private      3500    nl-BE  All time    82.60   
4          289658600.0      Private      3500    nl-BE  All time   204.30   
...                ...          ...       ...      ...       ...      ...   
5273       289757472.0      Private      3960    nl-BE  All time   277.50   
5274       289757472.0      Private      3960    nl-BE  All time  1017.65   
5275       289757472.0      Private      3960    nl-BE  All time     9.00   
5276       289757472.0      Private      3960    nl-BE  All time    34.80   
5277       289757472.0      Private      3960    nl-BE  

array([-4.13136511e-03,  1.25398350e-02, -7.72384949e+02,  0.00000000e+00])

In [ ]:
##########################
#Q3 Who are the employees?
##########################

In [97]:
#Q3.1 What are the routes of the employees?


In [ ]:
#Q3.2 What is the turnover for each employee?

In [ ]:
########################
#Q4 What are the routes?
########################
#Let's start by merging the df_RouteTemplate datdabse with the other two databases (CustomerType and Depotfor closer detail
#The merge is done via the equally named column "HVROUTETEMPLATE_NRID" in all databases.
#However the df_RouteTemplate contains these ID's in the first column, was this was the list of keys from our dictionary
#This results in the first column having no columname to merge with other databases. To solve this we can copy this column.
#After we copied this column we can add it to our original database, we now have duplicate column with which we can work with.
df_RouteTemplate["HVROUTETEMPLATE_NRID"] = df_RouteTemplate.index

#next we will merge the two databases df_RouteTemplate and df_CustomerType in order to link our customer and route info
df_CustomerType["CustomerID"]=df_CustomerType.index
df_Customer_Route = pd.merge(df_CustomerType, df_RouteTemplate, how = "outer", on = "HVROUTETEMPLATE_NRID")
#for future question we will also need the depot so that's why we join them as well
df_Customer_Route_Depot = pd.merge(df_Depot, df_Customer_Route, how = "outer", on = "HVROUTETEMPLATE_NRID")
df_CustomerType.head(5)
#df_Customer_Route.head(5)
#increasing the view of our dataframe, set the parameters if you want less rows to be visible
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
#Q4.1 How are the customers divided into regions?
#here we can see that every customer has its own routenumber, postcode, region, weekorder and dayofweek. 
#If we make a subselection of our total database and remove all duplicates for our region and postcode,
#we can find a database that shows us each region linked to each unique postcode, as we assumed (see picture of map)
#all unique postcodes are clustered in 4 groups en each cluster of postcodes corresponds to the same region!
#so to answer: The postcode of a customer determines its REGION
df_Customer_Route_Depot.drop_duplicates(subset = ["Postcode", "REGION"]).sort_values(by="REGION")

In [98]:
#Q4.2 Which customers are assigned to which routes?
df_Customer_Route_Depot.drop_duplicates(subset = ["HVROUTETEMPLATE_NRID", "WEEKORDER", "DAYOFWEEK_NRID"]).sort_values(by=["HVROUTETEMPLATE_NRID", "WEEKORDER", "DAYOFWEEK_NRID"])
df_Customer_Route_Depot.drop_duplicates().sort_values(by=["HVROUTETEMPLATE_NRID", "WEEKORDER", "DAYOFWEEK_NRID"])
df_CRV = pd.merge(df_CustomerType, df_Customer_Route_Depot, on="HVROUTETEMPLATE_NRID").drop(columns=["Postcode_x","CustomerID_x","CustomerType_x","Language_x","Season_x"])
df_CRV.sort_values(by=["CustomerID_y"])
df_CRV.head(5)
            

NameError: name 'df_Customer_Route_Depot' is not defined

In [ ]:
#Q4.3 Which routes are assigned to which depots?
#In the dataframe below we can see for each depot every single possible route, the total number of routes in the dataframe is
#39 which corresponds to the df_RouteTemplate dataframe, where there were 39 possible routes
#This shows that every route is linked to a certain depot
df_Customer_Route_Depot.drop_duplicates(subset = ["DEPOT", "HVROUTETEMPLATE_NRID"]).sort_values(by = "DEPOT")
len(df_Customer_Route_Depot.drop_duplicates(subset = ["DEPOT", "HVROUTETEMPLATE_NRID"]).sort_values(by = "DEPOT").index)

In [ ]:
############################################
#Q5 How can the company improve its service?
############################################

In [ ]:
#Q5.1 Which customers should be rewarded?

In [ ]:
#Q5.2 Which employees should be rewarded?

In [ ]:
#Q5.3 To which customers should the company send coupons in order to win them back?

In [ ]:
#Q5.4 Are there factors that the company can change in order to decrease the churning rate?

In [ ]:
#Q5.5 Would it be valuable to recommend (upsell / cross sell) products to a customer?

In [ ]:
#Q5.6 Which employees should be assigned to different routes?

In [ ]:
#Q5.7 Which routes should be reassigned to different depots?

In [ ]:
#Q5.8 Which customers should be reassigned to different routes?

In [ ]:
#Q5.9 Which depots should be removed? Where should the company create new depots?

In [ ]:
#Q5.10 Which products should be added / removed from depots?

In [ ]:
#Q5.11 Does customer satisfaction relate to different factors? Can the company respond to these
#factors?